In [1]:
import pandas as pd
# import scipy
import plotly.express as px
import pickle
from function import *
from sklearn.model_selection import train_test_split

parse_dates — распознавать ли даты, можно передать номера строк;  
date_parser — парсер дат;  

In [2]:
# %%time
# events = pd.read_csv('data/events.csv',
#                      parse_dates=['timestamp'],
#                      converters={'timestamp': lambda x: pd.to_datetime(x,unit='ms').strftime('%Y-%m-%d')})
# events.head()

In [3]:
# %%time
# events = pd.read_csv('data/events.csv',
#                      parse_dates=['timestamp'], 
#                      date_parser=lambda x: pd.to_datetime(x, unit='ms').strftime('%Y-%m-%d'))
# events.head()

In [4]:
events = pd.read_csv('data/events.csv')
events['timestamp'] = pd.to_datetime(events.timestamp,unit='ms',)#.dt.strftime('%Y-%m-%d')
events.head(1)

,timestamp,visitorid,event,itemid,transactionid
0,2015-06-02 05:02:12.117,257597,view,355908,NaN


Одно и тоже действие можно выполнить разными путями и затратить разное время и ресурсы! Как оказалость, самое простое и быстрое это сделать вручную, без применения параметров read_csv()

In [5]:
category_tree = pd.read_csv('data/category_tree.csv')
category_tree.head(1)

,categoryid,parentid
0,1016,213.0


In [6]:
item_properties_part1 = pd.read_csv('data/item_properties_part1.csv')
item_properties_part1.head(1)

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338


In [7]:
item_properties_part2 = pd.read_csv('data/item_properties_part2.csv')
item_properties_part2.head(1)

,timestamp,itemid,property,value
0,1433041200000,183478,561,769062


In [8]:
item_properties = pd.concat([item_properties_part1,item_properties_part2],ignore_index=True)
item_properties['timestamp']=pd.to_datetime(item_properties.timestamp,unit='ms',).dt.strftime('%Y-%m-%d')
item_properties.head(1)

,timestamp,itemid,property,value
0,2015-06-28,460429,categoryid,1338


In [9]:
item_properties.describe(include='all')

,timestamp,itemid,property,value
count,20275902,2.027590e+07,20275902,20275902
unique,18,NaN,1104,1966868
top,2015-05-10,NaN,888,769062
freq,3012756,NaN,3000398,1537247
mean,NaN,2.333904e+05,NaN,NaN
std,NaN,1.348452e+05,NaN,NaN
min,NaN,0.000000e+00,NaN,NaN
25%,NaN,1.165160e+05,NaN,NaN
50%,NaN,2.334830e+05,NaN,NaN
75%,NaN,3.503040e+05,NaN,NaN


Задание 2.1  
Сколько записей событий находится в датасете? Ответ введите без пробелов и знаков препинания.

In [10]:
events.shape[0]

2756101

Задание 2.2  
Какие типы событий содержатся в датасете? Выберите все верные варианты.

In [11]:
events.event.unique()

array(['view', 'addtocart', 'transaction'], dtype=object)

Задание 2.3  
Сколько уникальных признаков товара есть в датасете? Ответ вводите без пробелов и знаков препинания.

In [12]:
item_properties.property.nunique()

1104

Задание 2.4  
Анастасия: Кстати, слушай… Это, конечно, не моё дело, но мы же не можем оптимизировать оборот?  
Как думаешь, как мы будем измерять качество рекомендаций? У нас вроде только три места для показа товаров на главной странице.  
А ведь клиента интересует только продажи!  

Precision@3  

In [13]:
events = events.drop_duplicates()
item_properties = item_properties.drop_duplicates()
category_tree = category_tree.drop_duplicates()

Можешь, пожалуйста, прикинуть быстренько? Если мы всем будем рекомендовать топовые товары (по числу транзакций), то какое качество рекомендаций получится?

Короче, давай посчитаем такую штуку: возьмём статистику до 1 июля включительно и потом прикинем, как эти товары покупались после 1 июля. Мне нужно узнать, сколько процентов покупок мы можем так покрыть. 

Задание 4.1  
Какой процент продаж обеспечивают топовые товары (точка отсечения — 1 июля)? Для расчёта используйте топ-3 товаров по числу транзакций.

In [14]:
task_date = '2015-07-01'

print(events.timestamp.max())
print(events.timestamp.min())

2015-09-18 02:59:47.788000
2015-05-03 03:00:04.384000


In [15]:
# @Владислав_Масленников_ментор НЕ БЬЯСНИЛ ПОЧЕМУ ЭТО РЕШЕНИЕ НЕВЕРНОЕ! ПРОСТО КИНУЛ ГОТОВОЕ РЕШЕНИЕ И ВСЕ!!!
# Хотя все сделано в соответствии с заданием.

temp = events[(events.timestamp.dt.strftime('%Y-%m-%d') <= task_date)&(events.event == 'transaction')].groupby('itemid',as_index=False)['transactionid'].sum()
total_sales = temp['transactionid'].sum()
temp['procentage'] = temp['transactionid']/total_sales
top_3 = temp.nlargest(3, ['transactionid'], keep='first')['procentage'].sum() * 100
print("%.3f" % top_3)

0.999


Другой вариант решения!

In [16]:
temp = events[(events.timestamp.dt.strftime('%Y-%m-%d') <= task_date)&(events.event == 'transaction')].groupby('itemid',as_index=False)['transactionid'].count()
total_sales = temp['transactionid'].sum()
temp['transactionid'] = temp['transactionid']/total_sales
top_3 = temp.nlargest(3, ['transactionid'], keep='first')['transactionid'].sum() * 100
print("%.3f" % top_3)
top_list = temp.nlargest(3, ['transactionid'], keep='first')['itemid'].to_list()
top_list

0.985


[119736, 369447, 7943]

In [17]:
# как эти товары покупались после 1 июля.
temp = events[(events.timestamp.dt.strftime('%Y-%m-%d') > task_date)&(events.event == 'transaction')].groupby('itemid',as_index=False)['transactionid'].count()
total_sales = temp['transactionid'].sum()
temp['transactionid'] = temp['transactionid']/total_sales
display(temp[temp.itemid.isin(top_list)])

temp[temp.itemid.isin(top_list)]['transactionid'].sum() * 100

,itemid,transactionid
125,7943,0.001269
1925,119736,0.004838


0.610723350253807

In [18]:
item_properties

,timestamp,itemid,property,value
0,2015-06-28,460429,categoryid,1338
1,2015-09-06,206783,888,1116713 960601 n277.200
2,2015-08-09,395014,400,n552.000 639502 n720.000 424566
3,2015-05-10,59481,790,n15360.000
4,2015-05-17,156781,917,828513
...,...,...,...,...
20275897,2015-06-07,236931,929,n12.000
20275898,2015-08-30,455746,6,150169 639134
20275899,2015-08-16,347565,686,610834
20275900,2015-06-07,287231,867,769062


In [19]:
events['day_of_week'] = events['timestamp'].dt.weekday
events['year'] = events['timestamp'].dt.year
events['month'] = events['timestamp'].dt.month
events['day'] = events['timestamp'].dt.day
events['hour'] = events['timestamp'].dt.hour
events['minute'] = events['timestamp'].dt.minute

events.head()

,timestamp,visitorid,event,itemid,transactionid,day_of_week,year,month,day,hour,minute
0,2015-06-02 05:02:12.117,257597,view,355908,NaN,1,2015,6,2,5,2
1,2015-06-02 05:50:14.164,992329,view,248676,NaN,1,2015,6,2,5,50
2,2015-06-02 05:13:19.827,111016,view,318965,NaN,1,2015,6,2,5,13
3,2015-06-02 05:12:35.914,483717,view,253185,NaN,1,2015,6,2,5,12
4,2015-06-02 05:02:17.106,951259,view,367447,NaN,1,2015,6,2,5,2


Когда я училась в школе нас учили:

Утро: с 05 до 11 часов.

День: с 11 до 17 часов.

Вечер: с 17 до 23 часов.

Ночь: с 23 до 05 часов

В Америке обычно выделяют следующие части суток:

Утро: с 6 до 12 часов.

День: с 12 до 18 часов.

Вечер: с 18 до 22 часов.

Конечно, точные границы могут варьироваться в зависимости от индивидуальных привычек и регионов, но в общем, это самые распространенные временные интервалы. Какое время суток тебе больше всего нравится?


деление суток на утро день вечер в россии

В России, как правило, сутки делятся следующим образом:

Утро: с 6 до 12 часов.

День: с 12 до 18 часов.

Вечер: с 18 до 24 часов (полуночи).

Некоторые также выделяют ночь: с 24 часов до 6 утра. Это деление достаточно общее, и, конечно, может немного варьироваться в зависимости от личных предпочтений. В какое время суток ты чувствуешь себя наиболее продуктивно?


по данным этого сайта:
https://weather.fandom.com/ru/wiki/%D0%92%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%B0_%D1%81%D1%83%D1%82%D0%BE%D0%BA

Времена суток
Ниже представлена классификация времён суток.

Время суток	Период	Состояние неба
Ночь	С 22:00 по 4:59	Темно
Утро	С 5:00 по 11:59	Рассвет (Сумерки зимой)
День	С 12:00 по 16:59	Светло
Вечер	С 17:00 по 21:59	Сумерки
Данные времена суток имеют и более углублённую классификацию.

Например, ночь делится на дополуночную ночь (22:00-0:00), глубокую ночь (0:00-3:00) и предрассветную ночь (3:00-5:00).

Утро делится на раннее утро (5:00-8:00), среднее утро (8:00-10:00) и позднее утро (10:00-12:00).

День, в свою очередь, делится на полдень (12:00), середину дня (13:00-16:00) и поздний день (15:00-17:00).

Вечер-же делится на ранний вечер (с 17 по 19), центральный вечер (с 19:00 по 19:59) и поздний вечер (с 20 до 22-х часов).

In [20]:
events['part_day'] = events['hour'].apply(part_day)
events['part_day'].value_counts()

part_day
Evening      1092194
Night         887841
Afternoon     475533
Morning       300073
Name: count, dtype: int64

In [21]:
events['Day Period'] = events['hour'].apply(get_time_periods)
events['Day Period'].value_counts()

Day Period
Evening      1078018
Night         765788
Dawn          494507
Afternoon     293450
Morning       123878
Name: count, dtype: int64

In [22]:
train, test = train_test_split(events, test_size=0.3, shuffle=False)

In [23]:
test.timestamp.describe()

count                           826693
mean     2015-07-09 09:51:25.107366656
min         2015-05-24 06:00:15.401000
25%         2015-07-05 04:28:20.800000
50%      2015-07-14 05:16:27.512999936
75%         2015-07-23 20:16:30.376000
max         2015-08-01 05:59:58.954000
Name: timestamp, dtype: object

Только подскажи, пожалуйста, какое свойство айтемов не входит в топ-20?

In [24]:
item_properties.drop_duplicates(['itemid', 'property']).property.value_counts()[:20]

property
categoryid    417053
159           417053
283           417053
112           417053
764           417053
888           417053
364           417053
available     417053
790           417053
678           417019
917           416171
202           414217
6             409065
776           407305
839           396644
227           328096
698           274747
689           211791
28            169926
928           150121
Name: count, dtype: int64

In [25]:
# Эталонное решение
# Возьмем только самые распространенные proprties, например топ 20 (при построении модели можно играть)
top_properties = item_properties.drop_duplicates(['itemid', 'property']).groupby("property")['itemid'].count().sort_values(ascending=False)[:20]
top_properties

property
categoryid    417053
159           417053
790           417053
364           417053
112           417053
888           417053
764           417053
283           417053
available     417053
678           417019
917           416171
202           414217
6             409065
776           407305
839           396644
227           328096
698           274747
689           211791
28            169926
928           150121
Name: itemid, dtype: int64

In [26]:
# # Производим сериализацию и записываем результат в файл формата pkl
with open('data/events.pkl', 'wb') as output:
    pickle.dump(events, output)

with open('data/category_tree.pkl', 'wb') as output:
    pickle.dump(category_tree, output)

with open('data/item_properties.pkl', 'wb') as output:
    pickle.dump(item_properties, output)